# Importing the libraries

In [40]:
# import required packages

import pandas as pd
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
# nltk.download('punkt')
nltk.download('stopwords')
# nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Reading data files from drive

Reading Data from Drive

In [41]:
# Mounting drive
from google.colab import drive
drive.mount("/content/drive")

# reading data from drive
sql_lines_fuzzing=[]
f = open('/content/drive/MyDrive/Colab Notebooks/Sqli1.txt', "r")
for x in f:
    sql_lines_fuzzing.append(x)

    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data preprocessing

Data Preprocessing

In [42]:
# preprocess sql data to have same format for all files
def clean_sqli_data(data):
  for i in range(len(data)):
    data[i]=data[i].replace('\n', '')
    data[i]=data[i].replace('%20', ' ')
    data[i]=data[i].replace('=', ' = ')
    data[i]=data[i].replace('((', ' (( ')
    data[i]=data[i].replace('))', ' )) ')
    data[i]=data[i].replace('(', ' ( ')
    data[i]=data[i].replace(')', ' ) ')
  return data

In [43]:
# cleaning data from file1
sql_lines_fuzzing=clean_sqli_data(sql_lines_fuzzing)

In [44]:
# previewing the data
sql_lines_fuzzing[:10]

["'sqlvuln",
 "'+sqlvuln",
 'sqlvuln;',
 ' ( sqlvuln ) ',
 "a' or 1 = 1--",
 '"a"" or 1 = 1--"',
 ' or a  =  a',
 "a' or 'a'  =  'a",
 '1 or 1 = 1',
 "a' waitfor delay '0:0:10'--"]

Adding file 2 

In [45]:
# reading data from file2
sql_lines_bypass=[]
f = open('/content/drive/MyDrive/Colab Notebooks/sqli2.txt', "r")
for x in f:
    sql_lines_bypass.append(x)

In [46]:
sql_lines_bypass[:5] # previewing the data before cleaning

['&()o1: select * from users where id=1 or (\\)=1 union select 1,@@VERSION -- 1\n',
 '&(.)o: select * from users where id=1 or (\\.)=1 union select 1,@@VERSION -- 1\n',
 '&(1&1: select * from users where id=1 or (\\+)=1 or 1=1 -- 1\n',
 '&(1)o: select * from users where id=1 or (1)=1 union select 1,banner from v$version where rownum=1 -- 1\n',
 '&(1UE: select * from users where id=1 or (\\+)=1 union select 1,@@VERSION -- 1\n']

In [47]:
# cleaning the data
sql_lines_bypass=clean_sqli_data(sql_lines_bypass)

# removing unwanted symbols in the beginning of the sentences
for i in range(len(sql_lines_bypass)):
    sentence=sql_lines_bypass[i]
    sql_lines_bypass[i]=sentence.split(':')[1]

In [48]:
for i in range(5):
  print(sql_lines_bypass[:i])  # previewing the data after cleaning

[]
[' select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1']
[' select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1', ' select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1']
[' select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1', ' select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1', ' select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1']
[' select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1', ' select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1', ' select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1', ' select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1']


Adding file 3

In [49]:
# Adding file3
sql_lines_3=[]
f = open('/content/drive/MyDrive/Colab Notebooks/sqli_3.txt', "r")
for x in f:
    sql_lines_3.append(x)

# cleaning the data
sql_lines_3 = clean_sqli_data(sql_lines_3)

for i in range(6):
  print(sql_lines_3[:i])

[]
['or 1 = 1']
['or 1 = 1', 'or 1 = 1-- ']
['or 1 = 1', 'or 1 = 1-- ', 'or 1 = 1#']
['or 1 = 1', 'or 1 = 1-- ', 'or 1 = 1#', 'or 1 = 1/*']
['or 1 = 1', 'or 1 = 1-- ', 'or 1 = 1#', 'or 1 = 1/*', "admin' --"]


Adding file 4

In [50]:
# Adding file4(Non-sql injection data)
sql_lines_benign=[]
f = open('/content/drive/MyDrive/Colab Notebooks/sql_benign.txt', "r")
for x in f:
    sql_lines_benign.append(x)

In [51]:
# function to remove stopwords

stop_words = set(stopwords.words('english')) 
def fun_remove_stop_words(posts):
    filtered=''
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    
    return filtered

In [52]:
# convert from list to string

data=''
for x in sql_lines_benign:
    data+=" " + x

data=fun_remove_stop_words(data)  # remove stop words
data=data.split('.')              # split sentences

for i in range(len(data)):
    data[i]=data[i].replace('<', ' <')
    data[i]=data[i].replace('>', '> ')
    data[i]=data[i].replace('=', ' = ')  # seperate words inside the tags

benign_sentence=[]
for i in sql_lines_benign:
    sentences=i.split('.')
    
    for sentence in sentences:
        benign_sentence.append(sentence)


In [53]:
#previewing the data

print(benign_sentence[:5])
len(benign_sentence)

['Add plain text here\n', '“Ne te quaesiveris extra', '”\n', '“Man is his own star; and the soul that can\n', 'Render an honest and a perfect man,\n']


153

# Combining 3 files( combining different sqli attacks)

Combining 2 files(combining different sqli attacks)

In [54]:
all_sqli_sentence = sql_lines_fuzzing + sql_lines_bypass + sql_lines_3
len(all_sqli_sentence)

270

In [55]:
def optional_numeric_to_numeric(all_sqli_sentence):
    
    for i in range(len(all_sqli_sentence)):
        

        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1 ', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 1', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("'1 ", "'numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 1'", " numeric'")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1,', 'numeric,')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("1\ ", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("‘1", '‘numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 2 ", " numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3--', ' numeric--')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 4 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 5 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 6 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 7 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1234', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("22", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 200 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("23 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('"1', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1"', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("7659", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 37 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 45 ", ' numeric ')
    
    return all_sqli_sentence

In [56]:
print(all_sqli_sentence[:10])

["'sqlvuln", "'+sqlvuln", 'sqlvuln;', ' ( sqlvuln ) ', "a' or 1 = 1--", '"a"" or 1 = 1--"', ' or a  =  a', "a' or 'a'  =  'a", '1 or 1 = 1', "a' waitfor delay '0:0:10'--"]


In [57]:
# Adding labels to sqli data and benign data
# Add labels to sql data

values=[]
for i in all_sqli_sentence:
    values.append((i,1))

# give labels to benign data

for i in benign_sentence:
    values.append((i,0))


In [59]:
print(values[:6])

[("'sqlvuln", 1), ("'+sqlvuln", 1), ('sqlvuln;', 1), (' ( sqlvuln ) ', 1), ("a' or 1 = 1--", 1), ('"a"" or 1 = 1--"', 1)]


In [60]:
# converting to dataframe

df=pd.DataFrame(values,columns=['Sentence','Label'])

print(df.head)

<bound method NDFrame.head of                                           Sentence  Label
0                                         'sqlvuln      1
1                                        '+sqlvuln      1
2                                         sqlvuln;      1
3                                     ( sqlvuln )       1
4                                    a' or 1 = 1--      1
..                                             ...    ...
418           virtue in most request is conformity      0
419                  Self-reliance is its aversion      0
420                                 It loves not\n      0
421  realities and creators, but names and customs      0
422                                                     0

[423 rows x 2 columns]>


In [61]:
# saving data into csv format
df.to_csv('sqli.csv', index=False, encoding='utf-16')

#reading the data
df=pd.read_csv('sqli.csv',encoding='utf-16')

# Feature Extraction

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, max_features=600, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()

In [63]:
print(posts)
len(posts)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


423

In [64]:
x = posts
y = df['Label']

# Splitting the data for training process

In [65]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

# Model Testing

Model testing 1 (Ada Boost)


In [66]:
from sklearn.ensemble import AdaBoostClassifier
classifier1 = AdaBoostClassifier(n_estimators=60)
classifier1.fit(x_train,y_train)

# predicting the data
y_pred_AB = classifier1.predict(x_test)

# model accuracy testing
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred_AB))

0.8235294117647058


Model Testing 2 (Logistic Regression)

In [72]:
from sklearn.linear_model import LogisticRegression
classifier2 = LogisticRegression()
classifier2.fit(x_train,y_train)

# predicting the data
y_pred_LR = classifier2.predict(x_test)

# model accuracy testing
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred_LR))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_pred_AB,y_test))

0.8705882352941177
[[15  1]
 [14 55]]


Model Testing 3(Decision Tree)

In [68]:
from sklearn.tree import DecisionTreeClassifier
classifier3 = DecisionTreeClassifier(criterion='entropy')
classifier3.fit(x_train,y_train)

# predicting the data
y_pred_DT = classifier3.predict(x_test)

# model accuracy testing
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred_DT))

0.8352941176470589


Model Testing 4(Random Forest)

In [69]:
from sklearn.ensemble import RandomForestClassifier
classifier4 = RandomForestClassifier(n_estimators=80,criterion='entropy')
classifier4.fit(x_train,y_train)

# predicting the data
y_pred_RF = classifier4.predict(x_test)

# model accuracy testing
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred_RF))

0.8470588235294118


Model Testing 5(Naive Bayes)

In [70]:
from sklearn.naive_bayes import GaussianNB
classifier5 = GaussianNB()
classifier5.fit(x_train,y_train)


# predicting the data
y_pred = classifier5.predict(x_test)

# model accuracy testing
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.8705882352941177
